In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, \
  mean_absolute_error, r2_score, mean_squared_error
import matplotlib.pyplot as plt

In [2]:
df_returns = pd.DataFrame()

In [3]:
df_returns = pd.read_csv('S&P-returns.csv', index_col=0, parse_dates=True)
df = pd.read_csv('S&P100.csv', index_col=0, parse_dates=True)

#for name in df.columns:
#  df_returns[name] = np.log(df[name]).diff()

In [4]:
df.isna().sum().sum()

0

In [5]:
df_returns.head()

,AAPL,MSFT,AMZN,GOOGL,GOOG,PG,TSLA,BAC,NVDA,JNJ,UNH,JPM,OEF,SPY
Date,,,,,,,,,,,,,,
2012-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-04,0.005360,0.023261,-0.008526,0.004304,0.004304,-0.000449,-0.013264,0.001723,0.011332,-0.006090,0.014461,-0.000858,0.001721,0.001567
2012-01-05,0.011041,0.010167,0.000563,-0.013969,-0.013969,-0.004201,-0.021522,0.082555,0.035286,-0.001223,0.006677,0.020672,0.002061,0.002659
2012-01-06,0.010400,0.015415,0.027763,-0.013736,-0.013736,-0.002408,-0.007773,-0.020817,-0.011624,-0.008754,0.003606,-0.009009,-0.002577,-0.002581
2012-01-09,-0.001587,-0.013250,-0.022428,-0.043324,-0.043324,0.004211,0.012556,0.014458,0.000000,0.001541,-0.001137,-0.001698,0.001891,0.002424


In [6]:
df_returns['SPY'] = df_returns['SPY'].shift(-1)
df_returns['SPY'].tail()

Date
2022-01-19   -0.011130
2022-01-20   -0.019826
2022-01-21    0.004238
2022-01-24   -0.012284
2022-01-25         NaN
Name: SPY, dtype: float64

In [7]:
Ntest = 1000
train = df_returns.iloc[1:-Ntest]
test = df_returns.iloc[-Ntest:-1]

In [8]:
df_returns.columns

Index(['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'GOOG', 'PG', 'TSLA', 'BAC', 'NVDA',
       'JNJ', 'UNH', 'JPM', 'OEF', 'SPY'],
      dtype='object')

In [9]:

# x_cols = df.columns.drop('OEF')
x_cols = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 
         'GOOG', 'PG', 'TSLA', 'BAC', 
         'NVDA', 'JNJ', 'UNH', 'JPM']
x_cols

['AAPL',
 'MSFT',
 'AMZN',
 'GOOGL',
 'GOOG',
 'PG',
 'TSLA',
 'BAC',
 'NVDA',
 'JNJ',
 'UNH',
 'JPM']

In [10]:
Xtrain = train[x_cols]
Ytrain = train['SPY']
Xtest = test[x_cols]
Ytest = test['SPY']

In [11]:
Ytest.head()

Date
2018-02-06   -0.005440
2018-02-07   -0.038230
2018-02-08    0.014910
2018-02-09    0.014578
2018-02-12    0.002484
Name: SPY, dtype: float64

In [12]:
Xtrain.head()

,AAPL,MSFT,AMZN,GOOGL,GOOG,PG,TSLA,BAC,NVDA,JNJ,UNH,JPM
Date,,,,,,,,,,,,
2012-01-04,0.005360,0.023261,-0.008526,0.004304,0.004304,-0.000449,-0.013264,0.001723,0.011332,-0.006090,0.014461,-0.000858
2012-01-05,0.011041,0.010167,0.000563,-0.013969,-0.013969,-0.004201,-0.021522,0.082555,0.035286,-0.001223,0.006677,0.020672
2012-01-06,0.010400,0.015415,0.027763,-0.013736,-0.013736,-0.002408,-0.007773,-0.020817,-0.011624,-0.008754,0.003606,-0.009009
2012-01-09,-0.001587,-0.013250,-0.022428,-0.043324,-0.043324,0.004211,0.012556,0.014458,0.000000,0.001541,-0.001137,-0.001698
2012-01-10,0.003574,0.003598,0.004359,0.001092,0.001092,-0.004663,0.013487,0.055828,-0.004135,0.004150,-0.001899,0.021024


## Buy & Hold

In [13]:
#std buy-and-hold train and test
Ytrain.std(), Ytest.std()

(0.007692028195000439, 0.01323862118085596)

In [14]:
#SR buy-and-hold train and test
Ytrain.mean()/Ytrain.std(), Ytest.mean()/Ytest.std()

(0.06326366009989662, 0.03621311865338717)

In [15]:
# Total return buy-and-hold train and test
Ytrain.sum(), Ytest.sum()

(0.7455108132416282, 0.47893234786995187)

## XGBoost Regressor

In [16]:
from xgboost.sklearn import XGBRegressor
model = XGBRegressor()
model.fit(Xtrain, Ytrain)
model.score(Xtrain, Ytrain), model.score(Xtest, Ytest)

(0.9834298244484956, -0.22952952634520707)

In [17]:
# Direction
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
np.mean(np.sign(Ptrain) == np.sign(Ytrain)), np.mean(np.sign(Ptest) == np.sign(Ytest))

(0.9556135770234987, 0.4954954954954955)

In [18]:
set(np.sign(Ptrain)), set(np.sign(Ptest))

({-1.0, 1.0}, {-1.0, 1.0})

In [19]:
train_idx = df.index <= train.index[-1]
test_idx = df.index > train.index[-1]

train_idx[0] = False
test_idx[-1] = False

In [20]:
df_returns['Position'] = 0 # create new column
df_returns.loc[train_idx,'Position'] = (Ptrain > 0)
df_returns.loc[test_idx,'Position'] = (Ptest > 0)

In [21]:
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['SPY']

In [22]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

4.548949296737919

In [23]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.14035455660668728

In [24]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.004580387789241789, 0.6482612803385607)

In [25]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.010192066823229949, 0.01378474592985661)

In [26]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(9.797725547231596e-07, 0.00021527298153112607)

In [27]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.0009898346097824422, 0.01467218393870272)

In [28]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.0007017330758371461, 0.009170715330747646)

In [29]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(0.9834298244484956, -0.22952952634520707)

In [30]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(4435706936.555755, 7874461878.575875)

In [31]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

0.3175186381735368

In [32]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.4936500202014384

In [33]:
df_returns['Position'].to_csv('XGBRBool.csv', index = True)
sum(df_returns['Position'])

1397

## LGBM Regressor

In [34]:
from lightgbm import LGBMRegressor
model = LGBMRegressor()
model.fit(Xtrain, Ytrain)
model.score(Xtrain, Ytrain), model.score(Xtest, Ytest)

(0.8017931880447143, -0.12306299467737114)

In [35]:
# Direction
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
np.mean(np.sign(Ptrain) == np.sign(Ytrain)), np.mean(np.sign(Ptest) == np.sign(Ytest))

(0.8720626631853786, 0.4934934934934935)

In [36]:
set(np.sign(Ptrain)), set(np.sign(Ptest))

({-1.0, 1.0}, {-1.0, 1.0})

In [37]:
train_idx = df.index <= train.index[-1]
test_idx = df.index > train.index[-1]

train_idx[0] = False
test_idx[-1] = False

In [38]:
df_returns['Position'] = 0 # create new column
df_returns.loc[train_idx,'Position'] = (Ptrain > 0)
df_returns.loc[test_idx,'Position'] = (Ptest > 0)

In [39]:
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['SPY']

In [40]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

4.305100271239781

In [41]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.07023256140329881

In [42]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.0046854750974739895, 0.5997508496069112)

In [43]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.009839526139452895, 0.007144944103118711)

In [44]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(1.171970652388961e-05, 0.00019663221917908948)

In [45]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.0034234056908128215, 0.014022561077745016)

In [46]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.0024664824114634812, 0.00879838391446883)

In [47]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(0.8017931880447143, -0.12306299467737114)

In [48]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(16922369305.898806, 8807417389.688313)

In [49]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

0.7768947181720182

In [50]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.5096297406108135

In [51]:
df_returns['Position'].to_csv('LGBMRBool.csv', index = True)
sum(df_returns['Position'])

1419

In [52]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score

## XGBoost Classifier

In [53]:
from xgboost.sklearn import XGBClassifier
model = XGBClassifier()
Ctrain = (Ytrain > 0)
Ctest = (Ytest > 0)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

C:\Users\rapha\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:49:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(1.0, 0.5135135135135135)

In [54]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['SPY']
set(Ptrain), set(Ptest)

({False, True}, {False, True})

In [55]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

4.57496036184191

In [56]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.36037509518116373

In [57]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.0045681386530458815, 0.6537162686023011)

In [58]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.010446400949207927, 0.034532068294729566)

In [59]:
accuracy_score(Ctrain,Ptrain), accuracy_score(Ctest,Ptest)

(1.0, 0.5135135135135135)

In [60]:
precision_score(Ctrain,Ptrain), precision_score(Ctest,Ptest)

(1.0, 0.5652920962199313)

In [61]:
f1_score(Ctrain,Ptrain), f1_score(Ctest,Ptest)

(1.0, 0.5751748251748251)

In [62]:
roc_auc_score(Ctrain,Ptrain), roc_auc_score(Ctest,Ptest)

(1.0, 0.5032309421239932)

In [63]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(0.542389704434735, 0.5820364264104364)

In [64]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.7364711158183564, 0.7629131185203439)

In [65]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.5478162462054559, 0.5854330539791581)

In [66]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(-9172.039779984092, -3323.2953509082545)

In [67]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(528.804084883318, 4508107735442.251)

In [68]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

1.9881180951400796

In [69]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.9893955042924054

In [70]:
df_returns['Position'].to_csv('XGBCBool.csv', index = True)
sum(df_returns['Position'])

1422

## LGBM Classifier

In [71]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()
Ctrain = (Ytrain > 0)
Ctest = (Ytest > 0)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

(0.9980417754569191, 0.5395395395395396)

In [72]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['SPY']
set(Ptrain), set(Ptest)

({False, True}, {False, True})

In [73]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

4.54881261531812

In [74]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.4813063796443311

In [75]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.004553608757925888, 0.6520540065933288)

In [76]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.009690811460031009, 0.04971597784140582)

In [77]:
accuracy_score(Ctrain,Ptrain), accuracy_score(Ctest,Ptest)

(0.9980417754569191, 0.5395395395395396)

In [78]:
precision_score(Ctrain,Ptrain), precision_score(Ctest,Ptest)

(0.9976218787158145, 0.5841584158415841)

In [79]:
f1_score(Ctrain,Ptrain), f1_score(Ctest,Ptest)

(0.9982153480071387, 0.6061643835616438)

In [80]:
roc_auc_score(Ctrain,Ptrain), roc_auc_score(Ctest,Ptest)

(0.9979596751995596, 0.5266170997662809)

In [81]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(0.5430765813884214, 0.6058183457608605)

In [82]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.7369372981390082, 0.7783433341147469)

In [83]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.5484944585601041, 0.6094986014115925)

In [84]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(-9183.656426776248, -3459.125550436935)

In [85]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(529.1653759644299, 4508107735459.109)

In [86]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

1.988168291291101

In [87]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.9894553751092587

In [88]:
df_returns['Position'].to_csv('LGBMCBool.csv', index = True)
sum(df_returns['Position'])

1447

## KNN

In [89]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
Ctrain = (Ytrain > 0)
Ctest = (Ytest > 0)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

(0.6906005221932114, 0.5165165165165165)

In [90]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['SPY']
set(Ptrain), set(Ptest)

({False, True}, {False, True})

In [91]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

1.863627676840613

In [92]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.24670659878196854

In [93]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.005571023038250276, 0.2183561517575051)

In [94]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.009415430081206775, 0.02622860030868084)

In [95]:
accuracy_score(Ctrain,Ptrain), accuracy_score(Ctest,Ptest)

(0.6906005221932114, 0.5165165165165165)

In [96]:
precision_score(Ctrain,Ptrain), precision_score(Ctest,Ptest)

(0.700218818380744, 0.5706618962432916)

In [97]:
f1_score(Ctrain,Ptrain), f1_score(Ctest,Ptest)

(0.7297605473204104, 0.5691347011596788)

In [98]:
roc_auc_score(Ctrain,Ptrain), roc_auc_score(Ctest,Ptest)

(0.6829755023396642, 0.5092082868473986)

In [99]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(0.594232175302883, 0.5592409679447692)

In [100]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.7708645635277853, 0.7478241557644211)

In [101]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.597648693195745, 0.5628696436713001)

In [102]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(-10048.8135160597, -3193.0993130647353)

In [103]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(5879372882254.42, 9016215470510.89)

In [104]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

1.9911688689629294

In [105]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.9902954863003026

In [106]:
df_returns['Position'].to_csv('KNNBool.csv', index = True)
sum(df_returns['Position'])

1473

## Naive Bayes Gaussian Classifier

In [107]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
Ctrain = (Ytrain > 0)
Ctest = (Ytest > 0)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

(0.5489556135770235, 0.5285285285285285)

In [108]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['SPY']
set(Ptrain), set(Ptest)

({False, True}, {False, True})

In [109]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

0.8384715680168799

In [110]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.7350239418040294

In [111]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.00643895503143321, 0.0849990718033611)

In [112]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.009876827557135549, 0.07449352509693083)

In [113]:
accuracy_score(Ctrain,Ptrain), accuracy_score(Ctest,Ptest)

(0.5489556135770235, 0.5285285285285285)

In [114]:
precision_score(Ctrain,Ptrain), precision_score(Ctest,Ptest)

(0.5685372585096596, 0.566813509544787)

In [115]:
f1_score(Ctrain,Ptrain), f1_score(Ctest,Ptest)

(0.6414115204981838, 0.6210780370072405)

In [116]:
roc_auc_score(Ctrain,Ptrain), roc_auc_score(Ctest,Ptest)

(0.5289843104872006, 0.5058877659877685)

In [117]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(0.7084947811890757, 0.6803854777685489)

In [118]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.8417213203840542, 0.8248548222375552)

In [119]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.7105498260922937, 0.6838024777295859)

In [120]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(-11981.253274021241, -3885.014994835592)

In [121]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(8819059323202.854, 9016215470664.955)

In [122]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

1.9911293283562363

In [123]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.9879775935740793

In [124]:
df_returns['Position'].to_csv('NBGCBool.csv', index = True)
sum(df_returns['Position'])

1768